## MVP

In this MVP, I will show the workflow of what I will accomplish to setup a deployed system in AWS. I am using a simplified regression model using features that predict housing prices.

In [54]:
import sagemaker
import boto3
import os
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session
import json
import pandas as pd
import io

To see the objects in the S3 bucket, open up an s3 resrouce and print all objects from the bucket

In [55]:
s3 = boto3.resource("s3")
bucket = s3.Bucket(name="housepricesregression334")
for obj in bucket.objects.all():
    print(obj.key)


sample_submission.csv
saved_models/x.csv
test.csv
train.csv


From the s3 bucket, load the training dataset

In [56]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket='housepricesregression334', Key='train.csv')
train_df = pd.read_csv(io.BytesIO(obj['Body'].read()))


In [57]:
train_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


### Train the model

Training can be done similarly as you would in an offline model

In [58]:
X = train_df[['LotArea', 'MSSubClass', 'MoSold']]

In [59]:
y = train_df[['SalePrice']]

In [60]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.2, random_state = 42)

In [61]:
from sklearn.linear_model import LinearRegression

In [62]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [63]:
model.score(X_train, y_train)


0.07576954712652406

### Putting trained model into S3 Bucket

After the model has been trained, we want to save it back to the S3 bucket

In [69]:
bucket_name = "housepricesregression334"

prefix = 'saved_models2'

In [70]:
output = 's3://{}/{}'.format(bucket_name, prefix)
output

's3://housepricesregression334/saved_models2'

In [67]:
file_name = output + model_1.p'

In [ ]:
s3_resource = boto3.resource('s3')

bucket='housepricesregression334'
key= 'model_1.pkl'

pickle_byte_obj = pickle.dumps(model)

s3_resource.Object(bucket,key).put(Body=pickle_byte_obj)
